In [1]:
pip install pygad


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 1.5 MB/s eta 0:00:00


# solar 데이터

# L1-metric

In [2]:
import pandas as pd
import numpy as np
import pygad

In [3]:
data = pd.read_csv("solar.txt")
data.head
data=data.rename(columns={'cloud cover':'sky_cover'})

## M에 대한 회귀계수 구하는 유전자 알고리즘 함수

In [20]:
def ga_m(data):
  # 삼각퍼지화
  te=data['temp']
  spreadx= [None] * 1332
  for i in range(0,1331):
    spreadx[i]= (np.abs(te[i+1]-te[i]))/2
  spreadx[1331]=spreadx[1330]
  x1= te-spreadx
  x2= te+spreadx
  xdf=pd.DataFrame({'a':x1,'b':te,'c':x2})
  xa=xdf[xdf['a']< 0]
  xdf.loc[xa.index,'a']=0
  x=xdf.to_numpy()

  hu=data['humidity']
  spreadm= [None] * 1332
  for i in range(0,1331):
    spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
  spreadm[1331]=spreadm[1330]
  m1= hu-spreadm
  m2= hu+spreadm
  mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
  ma=mdf[mdf['a']< 0]
  mdf.loc[ma.index,'a']=0
  m=mdf.to_numpy()


  X_l=x[:,0]
  M_l=m[:,0]

  X_m=x[:,1]
  M_m=m[:,1]

  X_r=x[:,2]
  M_r=m[:,2]

  # 유전자 알고리즘 세팅
  num_generations = 150
  num_parents_mating = 5

  sol_per_pop = 100
  num_parameters = 2

  init_range_low = -1
  init_range_high = 1

  parent_selection_type = "sss"
  keep_parents = 5

  crossover_type = 'single_point'

  mutation_type = "random"
  mutation_percent_genes = 10

  # 왼
  def fitness_function_factory(X=X_l, M=M_l, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X + solution[0]
          errors = np.abs(pred - M)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_l=lr_ga.best_solution()[0]

  # 중
  def fitness_function_factory(X=X_m, M=M_m, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = np.abs(pred - M)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_m=lr_ga.best_solution()[0]

  # 오
  def fitness_function_factory(X=X_r, M=M_r, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = np.abs(pred - M)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_r=lr_ga.best_solution()[0]
  sol= (sol_l + sol_m + sol_r)/3
  return sol

In [21]:
sol_m=ga_m(data)
sol_m

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([ 0.87631533, -0.4912688 ])

## Y에 대한 회귀계수 구하는 유전자 알고리즘 함수

In [16]:
def ga_y(data):
  # 삼각퍼지화
  te=data['temp']
  spreadx= [None] * 1332
  for i in range(0,1331):
    spreadx[i]= (np.abs(te[i+1]-te[i]))/2
  spreadx[1331]=spreadx[1330]
  x1= te-spreadx
  x2= te+spreadx
  xdf=pd.DataFrame({'a':x1,'b':te,'c':x2})
  xa=xdf[xdf['a']< 0]
  xdf.loc[xa.index,'a']=0
  x=xdf.to_numpy()

  hu=data['humidity']
  spreadm= [None] * 1332
  for i in range(0,1331):
    spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
  spreadm[1331]=spreadm[1330]
  m1= hu-spreadm
  m2= hu+spreadm
  mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
  ma=mdf[mdf['a']< 0]
  mdf.loc[ma.index,'a']=0
  m=mdf.to_numpy()



  power=data['day_power']
  ydf=pd.DataFrame({'a':power,'b':power,'c':power})
  y= ydf.to_numpy()
  # 조절변수
  cl=data['sky_cover']
  w1= cl-1
  w2= cl+1
  wdf=pd.DataFrame({'a':w1,'b':cl,'c':w2})
  wa=wdf[wdf['a']< 1]
  wdf.loc[wa.index,'a']=1
  wc=wdf[wdf['c']> 8]
  wdf.loc[wc.index,'c']=8
  w=wdf.to_numpy()

  xw=x*w

  X_l=np.column_stack((x[:,0],w[:,0],xw[:,0],m[:,0]))
  Y_l=y[:,0]

  X_m=np.column_stack((x[:,1],w[:,1],xw[:,1],m[:,1]))
  Y_m=y[:,1]

  X_r=np.column_stack((x[:,2],w[:,2],xw[:,2],m[:,2]))
  Y_r=y[:,2]

  # 유전자 알고리즘 세팅
  num_generations = 600
  num_parents_mating = 10

  sol_per_pop = 400
  num_parameters = 5

  init_range_low = -1
  init_range_high = 1

  parent_selection_type = "sss"
  keep_parents = 10

  crossover_type = 'single_point'

  mutation_type = "random"
  mutation_percent_genes = 10

  # 왼
  def fitness_function_factory(X=X_l, Y=Y_l, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1] +solution[3]*X[:,2] +solution[4]*X[:,3] +solution[0]
          errors = np.abs(pred - Y)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_l=lr_ga.best_solution()[0]

  # 중
  def fitness_function_factory(X=X_m, Y=Y_m, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1] +solution[3]*X[:,2] +solution[4]*X[:,3] +solution[0]
          errors = np.abs(pred - Y)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_m=lr_ga.best_solution()[0]

  # 오
  def fitness_function_factory(X=X_r, Y=Y_r, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1] +solution[3]*X[:,2] +solution[4]*X[:,3] +solution[0]
          errors = np.abs(pred - Y)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_r=lr_ga.best_solution()[0]
  sol= (sol_l + sol_m + sol_r)/3
  return sol

In [17]:
sol=ga_y(data)
sol

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([ 0.64231219,  0.37201254, -0.01076961,  0.0301624 , -0.82542028])

In [18]:
te=data['temp']
spreadx= [None] * 1332
for i in range(0,1331):
  spreadx[i]= (np.abs(te[i+1]-te[i]))/2
spreadx[1331]=spreadx[1330]
x1= te-spreadx
x2= te+spreadx
xdf=pd.DataFrame({'a':x1,'b':te,'c':x2})
xa=xdf[xdf['a']< 0]
xdf.loc[xa.index,'a']=0
x=xdf.to_numpy()

hu=data['humidity']
spreadm= [None] * 1332
for i in range(0,1331):
  spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
spreadm[1331]=spreadm[1330]
m1= hu-spreadm
m2= hu+spreadm
mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
ma=mdf[mdf['a']< 0]
mdf.loc[ma.index,'a']=0
m=mdf.to_numpy()

# 종속변수
hu=data['humidity']
spreadm= [None] * 1332
for i in range(0,1331):
  spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
spreadm[1331]=spreadm[1330]
m1= hu-spreadm
m2= hu+spreadm
mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
ma=mdf[mdf['a']< 0]
mdf.loc[ma.index,'a']=0
m=mdf.to_numpy()

power=data['day_power']
ydf=pd.DataFrame({'a':power,'b':power,'c':power})
y= ydf.to_numpy()
# 조절변수
cl=data['sky_cover']
w1= cl-1
w2= cl+1
wdf=pd.DataFrame({'a':w1,'b':cl,'c':w2})
wa=wdf[wdf['a']< 1]
wdf.loc[wa.index,'a']=1
wc=wdf[wdf['c']> 8]
wdf.loc[wc.index,'c']=8
w=wdf.to_numpy()
ones=np.ones(shape=(1332,3))
def dc(a,b):
  c=a*b
  sum=c.sum()
  return sum

xw=x*w

In [22]:
m= sol_m[0] + sol_m[1] * x

###RMSE

In [23]:
y_h= sol[0] + sol[1] * x + sol[2] * w + sol[3] * xw + sol[4] * m
MSE = np.sqrt(((y-y_h)**2).sum(axis=1).sum()/1332)
print(MSE)

0.31887140746539056


###MAE

In [24]:
abs=np.abs(y-y_h)
mae=abs.sum(axis=1).sum()/1332
print(mae)

0.44166272644297516


# L2-metric

## M에 대한 회귀계수 구하는 유전자 알고리즘 함수

In [25]:
def ga(data):
  # 삼각퍼지화
  te=data['temp']
  spreadx= [None] * 1332
  for i in range(0,1331):
    spreadx[i]= (np.abs(te[i+1]-te[i]))/2
  spreadx[1331]=spreadx[1330]
  x1= te-spreadx
  x2= te+spreadx
  xdf=pd.DataFrame({'a':x1,'b':te,'c':x2})
  xa=xdf[xdf['a']< 0]
  xdf.loc[xa.index,'a']=0
  x=xdf.to_numpy()

  hu=data['humidity']
  spreadm= [None] * 1332
  for i in range(0,1331):
    spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
  spreadm[1331]=spreadm[1330]
  m1= hu-spreadm
  m2= hu+spreadm
  mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
  ma=mdf[mdf['a']< 0]
  mdf.loc[ma.index,'a']=0
  m=mdf.to_numpy()

  X_l=x[:,0]
  M_l=m[:,0]

  X_m=x[:,1]
  M_m=m[:,1]

  X_r=x[:,2]
  M_r=m[:,2]

  # 유전자 알고리즘 세팅
  num_generations = 150
  num_parents_mating = 5

  sol_per_pop = 100
  num_parameters = 2

  init_range_low = -1
  init_range_high = 1

  parent_selection_type = "sss"
  keep_parents = 5

  crossover_type = 'single_point'

  mutation_type = "random"
  mutation_percent_genes = 10

  # 왼
  def fitness_function_factory(X=X_l, M=M_l, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = ((pred - M)**2)/1332
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_l=lr_ga.best_solution()[0]

  # 중
  def fitness_function_factory(X=X_m, M=M_m, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = ((pred - M)**2)/1332
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_m=lr_ga.best_solution()[0]

  # 오
  def fitness_function_factory(X=X_r, M=M_r, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = ((pred - M)**2)/1332
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_r=lr_ga.best_solution()[0]
  sol= (sol_l + sol_m + sol_r)/3
  return sol

In [26]:
sol_m=ga(data)
sol_m

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([ 0.83157951, -0.42589984])

## Y에 대한 회귀계수 구하는 유전자 알고리즘 함수

In [27]:
def ga(data):
  # 삼각퍼지화
  te=data['temp']
  spreadx= [None] * 1332
  for i in range(0,1331):
    spreadx[i]= (np.abs(te[i+1]-te[i]))/2
  spreadx[1331]=spreadx[1330]
  x1= te-spreadx
  x2= te+spreadx
  xdf=pd.DataFrame({'a':x1,'b':te,'c':x2})
  xa=xdf[xdf['a']< 0]
  xdf.loc[xa.index,'a']=0
  x=xdf.to_numpy()

  hu=data['humidity']
  spreadm= [None] * 1332
  for i in range(0,1331):
    spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
  spreadm[1331]=spreadm[1330]
  m1= hu-spreadm
  m2= hu+spreadm
  mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
  ma=mdf[mdf['a']< 0]
  mdf.loc[ma.index,'a']=0
  m=mdf.to_numpy()

  # 종속변수
  hu=data['humidity']
  spreadm= [None] * 1332
  for i in range(0,1331):
    spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
  spreadm[1331]=spreadm[1330]
  m1= hu-spreadm
  m2= hu+spreadm
  mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
  ma=mdf[mdf['a']< 0]
  mdf.loc[ma.index,'a']=0
  m=mdf.to_numpy()

  power=data['day_power']
  ydf=pd.DataFrame({'a':power,'b':power,'c':power})
  y= ydf.to_numpy()
  # 조절변수
  cl=data['sky_cover']
  w1= cl-1
  w2= cl+1
  wdf=pd.DataFrame({'a':w1,'b':cl,'c':w2})
  wa=wdf[wdf['a']< 1]
  wdf.loc[wa.index,'a']=1
  wc=wdf[wdf['c']> 8]
  wdf.loc[wc.index,'c']=8
  w=wdf.to_numpy()

  xw=x*w

  X_l=np.column_stack((x[:,0],w[:,0],xw[:,0],m[:,0]))
  Y_l=y[:,0]

  X_m=np.column_stack((x[:,1],w[:,1],xw[:,1],m[:,1]))
  Y_m=y[:,1]

  X_r=np.column_stack((x[:,2],w[:,2],xw[:,2],m[:,2]))
  Y_r=y[:,2]

  # 유전자 알고리즘 세팅
  num_generations = 600
  num_parents_mating = 10

  sol_per_pop = 400
  num_parameters = 5

  init_range_low = -1
  init_range_high = 1

  parent_selection_type = "sss"
  keep_parents = 10

  crossover_type = 'single_point'

  mutation_type = "random"
  mutation_percent_genes = 10

  # 왼
  def fitness_function_factory(X=X_l, Y=Y_l, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1] +solution[3]*X[:,2] +solution[4]*X[:,3] +solution[0]
          errors = ((pred - Y)**2)/1332
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_l=lr_ga.best_solution()[0]

  # 중
  def fitness_function_factory(X=X_m, Y=Y_m, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1] +solution[3]*X[:,2] +solution[4]*X[:,3] +solution[0]
          errors = ((pred - Y)**2)/1332
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_m=lr_ga.best_solution()[0]

  # 오
  def fitness_function_factory(X=X_r, Y=Y_r, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1] +solution[3]*X[:,2] +solution[4]*X[:,3] +solution[0]
          errors = ((pred - Y)**2)/1332
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2000)

  lr_ga.run()

  sol_r=lr_ga.best_solution()[0]
  sol= (sol_l + sol_m + sol_r)/3
  return sol

In [28]:
sol=ga(data)
sol

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([ 0.62950879,  0.38555695, -0.01080983,  0.02876026, -0.79687103])

In [29]:
m= sol_m[0] + sol_m[1] * x

###RMSE

In [30]:
y_h= sol[0] + sol[1] * x + sol[2] * w + sol[3] * xw + sol[4] * m
MSE = np.sqrt(((y-y_h)**2).sum(axis=1).sum()/1332)
print(MSE)

0.31642049026293534


###MAE

In [31]:
abs=np.abs(y-y_h)
mae=abs.sum(axis=1).sum()/1332
print(mae)

0.44417594234355373


# solar boot

# team 데이터

In [33]:
data=pd.read_csv('teams.csv')

## M에 대한 회귀계수 구하는 유전자 알고리즘 함수

In [99]:
def ga(data):
  # 삼각퍼지화
  df=data['dysfunc']
  xdf=pd.DataFrame({'a':df-0.05,'b':df,'c':df+0.05})
  x=xdf.to_numpy()
  net=data['negtone']
  mdf=pd.DataFrame({'a':net-0.05,'b':net,'c':net+0.05})
  m=mdf.to_numpy()
  per=data['perform']
  ydf=pd.DataFrame({'a':per-0.05,'b':per,'c':per+0.05})
  y=ydf.to_numpy()
  # 분리
  X_l=x[:,0]
  M_l=m[:,0]

  X_m=x[:,1]
  M_m=m[:,1]

  X_r=x[:,2]
  M_r=m[:,2]

 # 유전자 알고리즘 세팅
  num_generations = 100
  num_parents_mating = 5

  sol_per_pop = 100
  num_parameters = 2

  init_range_low = -1
  init_range_high = 1

  parent_selection_type = "sss"
  keep_parents = 4

  crossover_type = "single_point"

  mutation_type = "random"
  mutation_percent_genes = 10
  # 왼
  def fitness_function_factory(X=X_l, M=M_l, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X  +solution[0]
          errors = np.abs(pred - M)            # LAD
          fitness = 1/(np.sum(errors) - desired_output)   # desired_output 0설정 > 1/ sum(error) 최대로 만드는 sol 찾기
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_l=lr_ga.best_solution()[0]

  # 중
  def fitness_function_factory(X=X_m, M=M_m, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = np.abs(pred - M)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_m=lr_ga.best_solution()[0]

 # 오
  def fitness_function_factory(X=X_r, M=M_r, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = np.abs(pred - M)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_r=lr_ga.best_solution()[0]

  sol= (sol_l + sol_m + sol_r)/3
  return sol

In [100]:
sol_m=ga(data)
sol_m

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([-0.03758967,  0.58164122])

## Y에 대한 회귀계수 구하는 유전자 알고리즘 함수

In [101]:
def ga(data):
  # 삼각퍼지화
  df=data['dysfunc']
  xdf=pd.DataFrame({'a':df-0.05,'b':df,'c':df+0.05})
  x=xdf.to_numpy()
  net=data['negtone']
  mdf=pd.DataFrame({'a':net-0.05,'b':net,'c':net+0.05})
  m=mdf.to_numpy()
  per=data['perform']
  ydf=pd.DataFrame({'a':per-0.05,'b':per,'c':per+0.05})
  y=ydf.to_numpy()
  # 분리
  X_l=np.column_stack((x[:,0],m[:,0]))
  Y_l=y[:,0]

  X_m=np.column_stack((x[:,1],m[:,1]))
  Y_m=y[:,1]

  X_r=np.column_stack((x[:,2],m[:,2]))
  Y_r=y[:,2]

 # 유전자 알고리즘 세팅
  num_generations = 100
  num_parents_mating = 5

  sol_per_pop = 200
  num_parameters = 3

  init_range_low = -1
  init_range_high = 1

  parent_selection_type = "sss"
  keep_parents = 4

  crossover_type = "single_point"

  mutation_type = "random"
  mutation_percent_genes = 10
  # 왼
  def fitness_function_factory(X=X_l, Y=Y_l, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1]  +solution[0]
          errors = np.abs(pred - Y)            # LAD
          fitness = 1/(np.sum(errors) - desired_output)   # desired_output 0설정 > 1/ sum(error) 최대로 만드는 sol 찾기
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_l=lr_ga.best_solution()[0]

  # 중
  def fitness_function_factory(X=X_m, Y=Y_m, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1]  +solution[0]
          errors = np.abs(pred - Y)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_m=lr_ga.best_solution()[0]

 # 오
  def fitness_function_factory(X=X_r, Y=Y_r, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1]  +solution[0]
          errors = np.abs(pred - Y)
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_r=lr_ga.best_solution()[0]

  sol= (sol_l + sol_m + sol_r)/3
  return sol

In [102]:
sol=ga(data)
sol

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([-0.03718357,  0.46413724, -0.53751859])

#삼각퍼지화

In [103]:
df=data['dysfunc']
xdf=pd.DataFrame({'a':df-0.05,'b':df,'c':df+0.05})
x=xdf.to_numpy()
net=data['negtone']
mdf=pd.DataFrame({'a':net-0.05,'b':net,'c':net+0.05})
m=mdf.to_numpy()
per=data['perform']
ydf=pd.DataFrame({'a':per-0.05,'b':per,'c':per+0.05})
y=ydf.to_numpy()

# RMSE

In [104]:
m= sol_m[0] + sol_m[1] * x

In [105]:
y_h= sol[0] + sol[1] * x + sol[2] * m
MSE = np.sqrt(((y-y_h)**2).sum(axis=1).sum()/60)
print(MSE)

0.8976497204036439


# MAE

In [106]:
abs=np.abs(y-y_h)
mae=abs.sum(axis=1).sum()/60
print(mae)

1.1932415220276114


# L2

In [107]:
data=pd.read_csv('teams.csv')

## M에 대한 회귀계수 구하는 유전자 알고리즘 함수

In [108]:
def ga(data):
  # 삼각퍼지화
  df=data['dysfunc']
  xdf=pd.DataFrame({'a':df-0.05,'b':df,'c':df+0.05})
  x=xdf.to_numpy()
  net=data['negtone']
  mdf=pd.DataFrame({'a':net-0.05,'b':net,'c':net+0.05})
  m=mdf.to_numpy()
  per=data['perform']
  ydf=pd.DataFrame({'a':per-0.05,'b':per,'c':per+0.05})
  y=ydf.to_numpy()
  # 분리
  X_l=x[:,0]
  M_l=m[:,0]

  X_m=x[:,1]
  M_m=m[:,1]

  X_r=x[:,2]
  M_r=m[:,2]

 # 유전자 알고리즘 세팅
  num_generations = 100
  num_parents_mating = 5

  sol_per_pop = 100
  num_parameters = 2

  init_range_low = -1
  init_range_high = 1

  parent_selection_type = "sss"
  keep_parents = 4

  crossover_type = "single_point"

  mutation_type = "random"
  mutation_percent_genes = 10
  # 왼
  def fitness_function_factory(X=X_l, M=M_l, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X  +solution[0]
          errors = ((pred - M)**2)/60          # LAD
          fitness = 1/(np.sum(errors) - desired_output)   # desired_output 0설정 > 1/ sum(error) 최대로 만드는 sol 찾기
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_l=lr_ga.best_solution()[0]

  # 중
  def fitness_function_factory(X=X_m, M=M_m, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = ((pred - M)**2)/60
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_m=lr_ga.best_solution()[0]

 # 오
  def fitness_function_factory(X=X_r, M=M_r, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X +solution[0]
          errors = ((pred - M)**2)/60
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_r=lr_ga.best_solution()[0]

  sol= (sol_l + sol_m + sol_r)/3
  return sol

In [109]:
sol_m=ga(data)
sol_m

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([0.02541218, 0.62017231])

## Y에 대한 회귀계수 구하는 유전자 알고리즘 함수

In [110]:
def ga(data):
  # 삼각퍼지화
  df=data['dysfunc']
  xdf=pd.DataFrame({'a':df-0.05,'b':df,'c':df+0.05})
  x=xdf.to_numpy()
  net=data['negtone']
  mdf=pd.DataFrame({'a':net-0.05,'b':net,'c':net+0.05})
  m=mdf.to_numpy()
  per=data['perform']
  ydf=pd.DataFrame({'a':per-0.05,'b':per,'c':per+0.05})
  y=ydf.to_numpy()
  # 분리
  X_l=np.column_stack((x[:,0],m[:,0]))
  Y_l=y[:,0]

  X_m=np.column_stack((x[:,1],m[:,1]))
  Y_m=y[:,1]

  X_r=np.column_stack((x[:,2],m[:,2]))
  Y_r=y[:,2]

 # 유전자 알고리즘 세팅
  num_generations = 100
  num_parents_mating = 5

  sol_per_pop = 200
  num_parameters = 3

  init_range_low = -1
  init_range_high = 1

  parent_selection_type = "sss"
  keep_parents = 4

  crossover_type = "single_point"

  mutation_type = "random"
  mutation_percent_genes = 10
  # 왼
  def fitness_function_factory(X=X_l, Y=Y_l, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1]  +solution[0]
          errors = ((pred - Y)**2)/60            # LAD
          fitness = 1/(np.sum(errors) - desired_output)   # desired_output 0설정 > 1/ sum(error) 최대로 만드는 sol 찾기
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_l=lr_ga.best_solution()[0]

  # 중
  def fitness_function_factory(X=X_m, Y=Y_m, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1]  +solution[0]
          errors = ((pred - Y)**2)/60
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_m=lr_ga.best_solution()[0]

 # 오
  def fitness_function_factory(X=X_r, Y=Y_r, desired_output = 0 ):

      def fitness_function(ga_instance,solution, solution_idx):

          pred = solution[1]*X[:,0] +solution[2]*X[:,1]  +solution[0]
          errors = ((pred - Y)**2)/60
          fitness = 1/(np.sum(errors) - desired_output)
          return fitness

      return fitness_function

  lr_ga = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_function_factory(),
                        sol_per_pop=sol_per_pop,
                        num_genes=num_parameters,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        keep_parents=keep_parents,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_percent_genes=mutation_percent_genes,
                        random_seed=2023)

  lr_ga.run()

  sol_r=lr_ga.best_solution()[0]

  sol= (sol_l + sol_m + sol_r)/3
  return sol

In [111]:
sol=ga(data)
sol

/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


array([-0.02136826,  0.44132241, -0.53520805])

In [112]:
df=data['dysfunc']
xdf=pd.DataFrame({'a':df-0.05,'b':df,'c':df+0.05})
x=xdf.to_numpy()
net=data['negtone']
mdf=pd.DataFrame({'a':net-0.05,'b':net,'c':net+0.05})
m=mdf.to_numpy()
per=data['perform']
ydf=pd.DataFrame({'a':per-0.05,'b':per,'c':per+0.05})
y=ydf.to_numpy()

In [113]:
m= sol_m[0] + sol_m[1] * x

###RMSE

In [114]:
y_h= sol[0] + sol[1] * x + sol[2] * m
MSE = np.sqrt(((y-y_h)**2).sum(axis=1).sum()/60)
print(MSE)

0.8968031643721802


###MAE

In [115]:
abs=np.abs(y-y_h)
mae=abs.sum(axis=1).sum()/60
print(mae)

1.1919200474329485


#부트스트랩

In [ ]:
from tqdm import tqdm

In [ ]:
c_boot=[None]*5000
b_boot=[None]*5000
bi_boot=[None]*5000
for i in tqdm(range(0,5000)):
  new_data=data.sample(n=60,replace=True,random_state=i)
  sol=ga(new_data)
  c_boot[i]=sol[1]
  b_boot[i]= sol[2]
  bi_boot[i]= sol[0]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 67%|██████▋   | 3334/5000 [2:42:08<1:18:12,  2.82s/it]/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
 67%|██████▋   | 3335/5000 [2:42:12<1:26:53,  3.13s/it]/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to m

In [ ]:
np.save('bi_boot',bi_boot)
np.save('b_boot',b_boot)
np.save('c_boot',c_boot)


In [ ]:
b_b=np.load('b_boot.npy',allow_pickle=True)
bi_b=np.load('bi_boot.npy',allow_pickle=True)
c_b=np.load('c_boot.npy',allow_pickle=True)

In [ ]:
b=b_b.mean()
bi=bi_b.mean()
c=c_b.mean()

In [ ]:
y_h= bi + c * x + b * m
MSE = np.sqrt(((y-y_h)**2).sum(axis=1).sum()/57)
print(MSE)
abs=np.abs(y-y_h)
mae=abs.sum(axis=1).sum()/57
print(mae)

0.8081247660884163
1.1170555410892993
